#### Installations:


In [ ]:
!pip install -q transformers torch
!pip install -q transformers
!pip install -q transformers torch sklearn
!pip install -q pandas
!pip install -uqq transformers==2.8.0

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


#### Import nessacery liberary:


In [ ]:
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW, AutoModelForPreTraining
from transformers import BertModel, BertTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import torch
import pandas as pd
import csv
import nltk
import re
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


#### Tokenization and Input Formatting:

In [ ]:
#removing entity mentions (eg. @united) and some special character before tokenizing our text:
def text_preprocessing_bert(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text
    # Print sentence 0
#print('Original: ', X[0])
#print('Processed: ', text_preprocessing_bert(X[0]))

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing_bert(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,             # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )

        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from sklearn.model_selection import train_test_split

In [ ]:
data_emotion = pd.read_csv('/content/emotion_dataset.csv').reset_index(drop=True)
data_emotion

,Unnamed: 0,Emotion,Text,Clean_Text
0,0,neutral,Why ?,NaN
1,1,joy,Sage Act upgrade on my to do list for tommorow.,Sage Act upgrade list tommorow
2,2,sadness,ON THE WAY TO MY HOMEGIRL BABY FUNERAL!!! MAN ...,WAY HOMEGIRL BABY FUNERAL MAN HATE FUNERALS SH...
3,3,joy,Such an eye ! The true hazel eye-and so brill...,eye true hazel eyeand brilliant Regular feat...
4,4,joy,@Iluvmiasantos ugh babe.. hugggzzz for u .! b...,ugh babe hugggzzz u babe naamazed nga ako e...
...,...,...,...,...
34787,34787,surprise,@MichelGW have you gift! Hope you like it! It'...,gift Hope like it hand wear Itll warm Lol
34788,34788,joy,The world didnt give it to me..so the world MO...,world didnt meso world DEFINITELY cnt away
34789,34789,anger,A man robbed me today .,man robbed today
34790,34790,fear,"Youu call it JEALOUSY, I call it of #Losing YO...",Youu JEALOUSY #Losing YOU


In [ ]:
for text in data_emotion["Text"]:
    text_preprocessing_bert(text)

In [ ]:
emotions = data_emotion['Emotion'].unique()
emotions

array(['neutral', 'joy', 'sadness', 'fear', 'surprise', 'anger', 'shame',
       'disgust'], dtype=object)

In [ ]:
texts = data_emotion["Text"].tolist()
labels = data_emotion["Emotion"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [ ]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [ ]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
print(labels)
labels = label_encoder.fit_transform(labels)
print(labels)

['neutral', 'joy', 'sadness', 'joy', 'joy', 'fear', 'sadness', 'surprise', 'surprise', 'surprise', 'anger', 'sadness', 'joy', 'fear', 'anger', 'sadness', 'surprise', 'joy', 'joy', 'surprise', 'sadness', 'joy', 'fear', 'anger', 'fear', 'sadness', 'surprise', 'joy', 'sadness', 'joy', 'joy', 'fear', 'fear', 'shame', 'fear', 'joy', 'sadness', 'surprise', 'sadness', 'surprise', 'surprise', 'anger', 'fear', 'sadness', 'anger', 'fear', 'sadness', 'joy', 'anger', 'joy', 'joy', 'fear', 'sadness', 'sadness', 'sadness', 'sadness', 'fear', 'fear', 'sadness', 'sadness', 'surprise', 'neutral', 'joy', 'joy', 'surprise', 'fear', 'fear', 'fear', 'neutral', 'fear', 'joy', 'joy', 'fear', 'joy', 'sadness', 'fear', 'neutral', 'anger', 'fear', 'fear', 'fear', 'joy', 'anger', 'anger', 'joy', 'surprise', 'surprise', 'surprise', 'anger', 'surprise', 'disgust', 'anger', 'joy', 'fear', 'surprise', 'sadness', 'joy', 'anger', 'sadness', 'joy', 'joy', 'surprise', 'joy', 'fear', 'anger', 'joy', 'surprise', 'sadness'

In [ ]:
# Create a LabelEncoder object
# le = LabelEncoder()

# Fit the LabelEncoder to the labels
# le.fit(labels)

# Transform the labels to integers
# labels = le.transform(labels)

# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [ ]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])# loss="categorical_crossentropy"
model.fit(xtrain, ytrain, epochs=15, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/15
870/870 [==============================] - 20s 20ms/step - loss: 1.3694 - accuracy: 0.5034 - val_loss: 1.0987 - val_accuracy: 0.6242
Epoch 2/15
870/870 [==============================] - 6s 6ms/step - loss: 0.6191 - accuracy: 0.7974 - val_loss: 1.1083 - val_accuracy: 0.6328
Epoch 3/15
870/870 [==============================] - 4s 5ms/step - loss: 0.1922 - accuracy: 0.9446 - val_loss: 1.3587 - val_accuracy: 0.6219
Epoch 4/15
870/870 [==============================] - 4s 4ms/step - loss: 0.0661 - accuracy: 0.9841 - val_loss: 1.5700 - val_accuracy: 0.6157
Epoch 5/15
870/870 [==============================] - 4s 5ms/step - loss: 0.0338 - accuracy: 0.9924 - val_loss: 1.7872 - val_accuracy: 0.6155
Epoch 6/15
870/870 [==============================] - 4s 4ms/step - loss: 0.0248 - accuracy: 0.9944 - val_loss: 1.9462 - val_accuracy: 0.6091
Epoch 7/15
870/870 [==============================] - 3s 4ms/step - loss: 0.0212 - accuracy: 0.9950 - val_loss: 2.1488 - val_accuracy: 0.6077
Epoc

In [ ]:
input_text = "Nobody doubted him , or failed to recognise his temporary financial embarrassment .  "

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 93ms/step
['shame']


In [ ]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

# Assuming 'tokenizer', 'max_length', 'model', and 'label_encoder' are already defined and initialized

def classify_emotion(input_text):
    """Classify the emotion of the given input text."""
    # Preprocess the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
    prediction = model.predict(padded_input_sequence)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])[0]
    return predicted_label


def get_psychological_response(emotion):
    """Generate a psychological response based on the classified emotion."""
    responses = {
        'neutral': "Feeling neutral can be a peaceful place or a moment of rest. It’s okay to take this time just to be.",
        'joy': "It's wonderful to hear that you're feeling joyous! Cherish these moments—they're precious. Try to recognize what specifically is making you feel this joy, and consider how you might incorporate more of that into your daily routine.",
        'sadness': "It's completely normal to feel sad sometimes. t's completely normal to feel sad sometimes. Allow yourself to feel these emotions; you're not alone in this. Think about what might be causing your sadness and consider small actions that might brighten your day or provide comfort.; you're not alone in this. Think about what might be causing your sadness and consider some of these actions to help you feel better: take a walk, listen to your favorite music, or talk to someone you trust about how you feel.",
        'fear': "It can be really challenging to face fear. Remember, it’s okay to feel scared. Learning to view and cope with the feared object or situation differently can be transformative. Try to challenge your worries, tolerate uncomfortable feelings, and build confidence in managing your thoughts.",
        'surprise': "Life is full of surprises, and it’s alright to feel taken aback. Take a moment to steady yourself.",
        'anger': "Feeling anger is a natural response. In the heat of the moment, try this breathing technique: Stop and focus on your breath, breathe in for a count of 7, breathe out for a count of 11, and continue for at least a minute. This can help activate your 'rest and digest' system, clearing your mind and helping you respond from a relaxed place.",
        'shame': "Dealing with shame can be deeply challenging, but addressing the root causes of these feelings can be enlightening. Focus on understanding the connection between your thoughts, emotions, and behaviors. Consider the automatic thoughts that pass unnoticed, like 'I don’t deserve love if I make a mistake.' Learning to be more self-compassionate and less self-critical is a crucial step in this journey.",
        'disgust': "Disgust can be intense. It’s important to acknowledge how you feel, as this can lead to deeper insights."
    }
    return responses.get(emotion, "I'm here to support you, even if your feelings are hard to define right now.")


def respond_to_text(input_text):
    """Process the input text, classify its emotion, and return an appropriate psychological response."""
    emotion = classify_emotion(input_text)
    response = get_psychological_response(emotion)
    return response

# Example usage
input_text = "I am sad"
response = respond_to_text(input_text)
print(response)


1/1 [==============================] - 0s 20ms/step
It's completely normal to feel sad sometimes. t's completely normal to feel sad sometimes. Allow yourself to feel these emotions; you're not alone in this. Think about what might be causing your sadness and consider small actions that might brighten your day or provide comfort.; you're not alone in this. Think about what might be causing your sadness and consider some of these actions to help you feel better: take a walk, listen to your favorite music, or talk to someone you trust about how you feel.
